In [57]:
from datasets import Dataset
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
# https://data.stackexchange.com/travel/query/1869449/visa-question-and-answers

In [58]:
DATASET_PATH = "../Datasets/VIsa_Questions_Stack_Exchange_V2.csv"

In [59]:
dataset_df = pd.read_csv(DATASET_PATH, header=0)
dataset_df.head

<bound method NDFrame.head of        QScore                                          Post Link  \
0          30  {\r\n  "id": 11,\r\n  "title": "What is the be...   
1          30  {\r\n  "id": 11,\r\n  "title": "What is the be...   
2          30  {\r\n  "id": 11,\r\n  "title": "What is the be...   
3          30  {\r\n  "id": 11,\r\n  "title": "What is the be...   
4          25  {\r\n  "id": 136,\r\n  "title": "Is it safe to...   
...       ...                                                ...   
21048       1  {\r\n  "id": 191800,\r\n  "title": "Transit at...   
21049       0  {\r\n  "id": 191808,\r\n  "title": "Self trans...   
21050      -2  {\r\n  "id": 191821,\r\n  "title": "Can a Sche...   
21051       8  {\r\n  "id": 191834,\r\n  "title": "Japanese b...   
21052       8  {\r\n  "id": 191834,\r\n  "title": "Japanese b...   

                                                   Title  \
0      What is the best way to obtain visas for the T...   
1      What is the best way to ob

In [60]:
column_refined_df = dataset_df[["Tags", "Question Body", "Answer Body", "AScore"]]
# column_refined_df
max_score = np.max(dataset_df["AScore"])
# print(max_score)
def remove_html_tags(row):
    row["Question Body"] = BeautifulSoup(row['Question Body'], 'html.parser').get_text()
    row["Answer Body"] = BeautifulSoup(row['Answer Body'], 'html.parser').get_text()
    row["Tags"] = ",".join(row["Tags"][1:-1].split("><"))
    row["AScore"] /= max_score
    return row

column_refined_df = column_refined_df.apply(remove_html_tags, axis=1)
column_refined_df

,Tags,Question Body,Answer Body,AScore
0,"russia,visas,china,mongolia,trans-siberian",I'm planning on taking the trans-Siberian / tr...,"As I know, in Russia visas can be achieved in ...",0.048980
1,"russia,visas,china,mongolia,trans-siberian",I'm planning on taking the trans-Siberian / tr...,You're very unlikely to get them rejected if y...,0.118367
2,"russia,visas,china,mongolia,trans-siberian",I'm planning on taking the trans-Siberian / tr...,I was looking into the same journey recently a...,0.040816
3,"russia,visas,china,mongolia,trans-siberian",I'm planning on taking the trans-Siberian / tr...,"For the record, it is not necessary to go thro...",0.020408
4,"safety,transportation,middle-east,war-zones,yemen","I have been wanting to travel to Socotra, Yeme...",Answering my own question: it looks like all f...,0.069388
...,...,...,...,...
21048,"transit-visas,abu-dhabi,self-transfer",I have a flight(WizzAir) from Rome to Abu Dhab...,The UAE only allows transit visas if you are f...,0.008163
21049,"schengen,schengen-visas,layovers,self-transfer...",I’m flying from Dublin to Kochi with a self-tr...,You need a full Schengen visa (type C).\nAn Ai...,0.032653
21050,"visas,schengen,schengen-visas,seasonal",Do members of schengen always issue schengen v...,The whole principle of the Schengen Area is th...,0.008163
21051,"customs-and-immigration,japan",First time traveling to Japan and so far I've ...,"In a word, no. You are required to present you...",0.020408


In [61]:
# Preprocessing
hf_dataset = Dataset.from_pandas(column_refined_df)
print(hf_dataset)

Dataset({
    features: ['Tags', 'Question Body', 'Answer Body', 'AScore'],
    num_rows: 21053
})


In [62]:
hf_dataset = hf_dataset.train_test_split(test_size=0.25)
hf_dataset = hf_dataset.rename_column("Question Body", "question")
hf_dataset = hf_dataset.rename_column("Answer Body", "answer")
hf_dataset = hf_dataset.rename_column("AScore", "target_score")
hf_dataset = hf_dataset.rename_column("Tags", "meta_tags")
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['meta_tags', 'question', 'answer', 'target_score'],
        num_rows: 15789
    })
    test: Dataset({
        features: ['meta_tags', 'question', 'answer', 'target_score'],
        num_rows: 5264
    })
})

In [63]:
hf_dataset.save_to_disk("../Datasets/Visa_QA_V2")

Saving the dataset (1/1 shards): 100%|██████████| 5264/5264 [00:00<00:00, 116949.70 examples/s]
